In [51]:
import os
import re
import random
import glob
import pandas as pd
import numpy as np
from datetime import datetime
from IPython.display import display, clear_output, HTML
import base64
import io
import ipywidgets as widgets
from ipywidgets import interact, interact_manual
import base64
from github import Github
from github import InputGitTreeElement
import requests

In [52]:
%%javascript
//To remove scroll from any output area and automatically extend the jupyter cell
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [44]:
FLAG__ON_BINDER = False
box_binder = widgets.Checkbox(False, description='Running on binder')

def update_box_binder(change):
    global FLAG__ON_BINDER
    FLAG__ON_BINDER = change['new']

box_binder.observe(update_box_binder, 'value')

display(box_binder)

Checkbox(value=False, description='Running on binder')

In [45]:
if FLAG__ON_BINDER == True:
    ROOT_DIR = "/home/jovyan/cp_wssc"
else:
    ROOT_DIR = "."
    
print (ROOT_DIR)

.


In [46]:
TRAINING_DATA_DIR =  ROOT_DIR + "/Data/VALIDATED_DATA/Conventions/training_data/"

In [47]:
#Read training data file from github. 
url = 'https://raw.githubusercontent.com/aideenf/AIVC/master/cp_wssc/Data/VALIDATED_DATA/Conventions/training_data/training_data.csv'
convention_sentences_df = pd.read_csv(url, error_bad_lines=False)
#convention_sentences_df = pd.read_csv(TRAINING_DATA_DIR+'training_data.csv')
convention_sentences_df = convention_sentences_df.drop(convention_sentences_df.columns[0], axis=1)
row_to_edit = convention_sentences_df.sample(n = 1)
index_number = 0
display(convention_sentences_df.head(2))

#Save a backup of the training-file in Github


,sentence,provenance,convention_civic,convention_domestic,convention_green,convention_industrial,convention_inspired,convention_market,convention_project,convention_renown
0,in view of the issues with the swagger codegen...,github,1,0,0,0,0,0,0,0
1,this is an easter egg concerned with the com...,github,1,0,0,0,0,0,0,0


In [48]:
def time_stamp():
    now = datetime.now()
    timestamp = datetime.timestamp(now)
    dt_object = datetime.fromtimestamp(timestamp)
    words = str(dt_object).split(' ');
    return words[1]


def save_to_github(my_user, my_password, repo_url, my_file_list, my_file_name_list):

    user = my_user
    password = my_password
    url = repo_url
    try:
        g = Github(user,password)
    
        repo = g.get_user().get_repo(repo_url)

        file_list = my_file_list 
        file_names = my_file_name_list
                 
        commit_message = 'training data updated via the audit tool'
    
        master_ref = repo.get_git_ref('heads/master')
        master_sha = master_ref.object.sha
        base_tree = repo.get_git_tree(master_sha)
        element_list = list()

        for i, entry in enumerate(file_list):
            with open(entry) as input_file:
                data = input_file.read()
            if entry.endswith('.png'):
                data = base64.b64encode(data)
            element = InputGitTreeElement(file_names[i], '100644', 'blob', data)
            element_list.append(element)
        tree = repo.create_git_tree(element_list, base_tree)
        parent = repo.get_git_commit(master_sha)
        commit = repo.create_git_commit(commit_message, tree, [parent])
        master_ref.edit(commit.sha)
        print (commit)
    except:
        print("")
        print ("GITHUB SUBMIT FAILED:")
        print ("Are your github login credentials correct?")
        print ("Are you a collaberator in the repo?")



In [49]:
my_style = {'description_width': 'initial'}
my_layout = {'width': '500px'}
my_layout_short = {'width': '300px'}

selectHTML = widgets.HTML(
            "<h3><left>Apply Audit Filter</left></h3>")

space = widgets.Label('  ', layout=widgets.Layout(width='100%'))


convention_drop_down = widgets.Dropdown(
    options=['convention_civic', 'convention_domestic', 'convention_green', 'convention_industrial', 'convention_inspired', 'convention_market', 'convention_project', 'convention_renown'],
    value='convention_civic',
    description='Convention:',
    disabled=False,
    style = my_style,
    layout = my_layout
)

reclassify_drop_down = widgets.Dropdown(
    options=['flip current', 'convention_civic', 'convention_domestic', 'convention_green', 'convention_industrial', 'convention_inspired', 'convention_market', 'convention_project', 'convention_renown'],
    value='flip current',
    description='Re-Classify:',
    disabled=False,
    style = my_style,
    layout = my_layout
)

convention_true_false = widgets.Dropdown(
    options=['True', 'False'],
    value='True',
    description='Logged as:',
    disabled=False,
    style = my_style,
    layout = my_layout
)


confidence_score_slider = widgets.FloatSlider(
    min=0, 
    max=1, 
    step = 0.1,
    description='Confidence level:',
    style = my_style,
    layout = my_layout
    )


change_value_button = widgets.Button(
            description='Change Classification',
            disabled=False,
            button_style='',
            tooltip='Click me',
        )

audit_next_button = widgets.Button(
            description='Next',
            disabled=False,
            button_style='',
            tooltip='Click me',
        )


save_to_git_button = widgets.Button(
            description='Save to GitHub',
            disabled=False,
            button_style='',
            tooltip='Click me',
        )




user = widgets.Text(
            description='GIT username',
            disabled=False,
            layout=my_layout_short,
            tooltip='enter the new value name',
            style = my_style
        )


pswd = widgets.Password(
            description='GIT password',
            disabled=False,
            layout=my_layout_short,
            tooltip='enter the new value name',
            style = my_style
        )



def change_value_button_clicked(b):
    global index_number
    global row_to_edit
    global convention_sentences_df
    with output:
        current_value = row_to_edit[convention_drop_down.value].values.astype(int)
        if (current_value == 1) :
            new_value = 0
            print ("Changed from 1 to", new_value)
        elif (current_value == 0):
            new_value = 1
            print ("Changed from 0 to", new_value)
        convention_sentences_df[convention_drop_down.value][row_to_edit.index.values.astype(int)[0]] = new_value
        print ("change saved locally in data frame")
        #display(convention_sentences_df.loc[index_number , : ])
        

def save_changes_to_github_button_clicked(b):
    global convention_sentences_df
    GIT_USER = user.value
    GIT_PSWD = pswd.value
    file = "training_data.csv"
    save_file_as = 'cp_wssc/Data/VALIDATED_DATA/Conventions/training_data/' + file
    path = TRAINING_DATA_DIR + file
    git_repo = 'AIVC'
    print ("storing updated version to ", path)
    convention_sentences_df.to_csv(path)
    
    print ("pushing updated version to github ",git_repo, save_file_as)
    
    save_to_github(GIT_USER, 
                    GIT_PSWD,
                    git_repo,
                    [path],
                    [save_file_as])
  
    

def audit_next_button_clicked(b):
        with output:
            clear_output(wait=True)
            output_function(convention_drop_down.value, confidence_score_slider.value, convention_true_false.value )



def output_function(convention_drop_down_value, confidence_score_slider_value, convention_true_false_value ):
    global convention_sentences_df
    global row_to_edit
    global index_number
    convention = convention_drop_down_value.replace('convention_', '')
    
    if (convention_true_false_value == "True"):
        
        row_to_edit = convention_sentences_df.loc[convention_sentences_df[convention_drop_down_value] == 1].sample(n = 1)  
    
    elif (convention_true_false_value == "False"):
        row_to_edit = convention_sentences_df.loc[convention_sentences_df[convention_drop_down_value] == 0].sample(n = 1)  
    
    row_to_edit = row_to_edit[['sentence', convention_drop_down_value]]
    display (row_to_edit)
    print ("")
    sentence = row_to_edit['sentence'].values.astype(str)[0]
    display(HTML("<b>Sentence to review: </b><font color='green'>" + sentence + "</font>" ))
    display(HTML("<b>Classified as: </b><font color='green'> " + convention_true_false_value + "</font> for <font color='green'>" +  convention + " </font>economy of conventions" ))
    index_number = row_to_edit.index.values.astype(int)[0]
  
output = widgets.interactive_output(output_function, {
            'convention_drop_down_value': convention_drop_down,
            'confidence_score_slider_value': confidence_score_slider,
            'convention_true_false_value': convention_true_false
            })

change_value_button.on_click(change_value_button_clicked)
audit_next_button.on_click(audit_next_button_clicked)
save_to_git_button.on_click(save_changes_to_github_button_clicked)


In [50]:
display (
        space,
        selectHTML,
        convention_drop_down, 
        convention_true_false, 
        confidence_score_slider, 
        output, 
        space,
        reclassify_drop_down,
        widgets.HBox([change_value_button, audit_next_button]),
        space,
        widgets.HBox([save_to_git_button, user, pswd]),
    )

Label(value='  ', layout=Layout(width='100%'))

HTML(value='<h3><left>Apply Audit Filter</left></h3>')

Dropdown(description='Convention:', layout=Layout(width='500px'), options=('convention_civic', 'convention_dom…

Dropdown(description='Logged as:', layout=Layout(width='500px'), options=('True', 'False'), style=DescriptionS…

FloatSlider(value=0.0, description='Confidence level:', layout=Layout(width='500px'), max=1.0, style=SliderSty…

Output()

Label(value='  ', layout=Layout(width='100%'))

Dropdown(description='Re-Classify:', layout=Layout(width='500px'), options=('flip current', 'convention_civic'…

Label(value='  ', layout=Layout(width='100%'))

storing updated version to  ./Data/VALIDATED_DATA/Conventions/training_data/training_data.csv
pushing updated version to github  AIVC cp_wssc/Data/VALIDATED_DATA/Conventions/training_data/training_data.csv
GitCommit(sha="7e6f21203c2699208105842ce6858fb101780a2e")
storing updated version to  ./Data/VALIDATED_DATA/Conventions/training_data/training_data.csv
pushing updated version to github  AIVC cp_wssc/Data/VALIDATED_DATA/Conventions/training_data/training_data.csv
GitCommit(sha="847d309cc48495216f174f089c5d421de70da1ba")
